In [ ]:
import os
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap as lsc
import torch
import h5py

from deeplsd.utils.tensor import batch_to_device
from deeplsd.models.deeplsd_inference import DeepLSD
# from deeplsd.geometry.viz_2d import plot_images, plot_lines

plt.rcParams.update({'axes.titlesize': 5})

In [ ]:
def plot_images(imgs, titles=None, cmaps='gray', dpi=300, size=3, pad=0):
    """Plot a set of images horizontally.
    Args:
        imgs: a list of NumPy or PyTorch images, RGB (H, W, 3) or mono (H, W).
        titles: a list of strings, as titles for each image.
        cmaps: colormaps for monochrome images.
    """
    n = len(imgs)
    if not isinstance(cmaps, (list, tuple)):
        cmaps = [cmaps] * n
    figsize = (size*n, size*3/4) if size is not None else None
    fig, ax = plt.subplots(1, n, figsize=figsize, dpi=dpi)
    if n == 1:
        ax = [ax]
    for i in range(n):
        ax[i].imshow(imgs[i], cmap=plt.get_cmap(cmaps[i]))
        ax[i].get_yaxis().set_ticks([])
        ax[i].get_xaxis().set_ticks([])
        ax[i].set_axis_off()
        for spine in ax[i].spines.values():  # remove frame
            spine.set_visible(False)
        # if titles:
            # ax[i].set_title(titles[i], fontsize=12)
    fig.tight_layout(pad=pad)
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)  # Remove spacing

        
        
def plot_lines(lines, line_colors='orange', point_color='cyan',
               ps=1, lw=0.5, indices=(0, 1), alpha=1):
    """ Plot lines and endpoints for existing images.
    Args:
        lines: list of ndarrays of size (N, 2, 2).
        line_colors: string, or list of list of tuples (one for per line).
        point_color: unique color for all endpoints.
        ps: size of the keypoints as float pixels.
        lw: line width as float pixels.
        indices: indices of the images to draw the matches on.
        alpha: alpha transparency.
    """
    if not isinstance(line_colors, list):
        line_colors = [[line_colors] * len(l) for l in lines]
    for i in range(len(lines)):
        if ((not isinstance(line_colors[i], list))
            and (not isinstance(line_colors[i], np.ndarray))):
            line_colors[i] = [line_colors[i]] * len(lines[i])

    fig = plt.gcf()
    # fig.set_size_inches(6, 4) 
    ax = fig.axes
    assert len(ax) > max(indices)
    axes = [ax[i] for i in indices]
    fig.canvas.draw()

    # Plot the lines and junctions
    for a, l, lc in zip(axes, lines, line_colors):
        for i in range(len(l)):
            # x1, x2 = (l[i, 0, 0], l[i, 1, 0])
            # y1, y2 = (l[i, 0, 1], l[i, 1, 1])
            line = matplotlib.lines.Line2D(
                (l[i, 0, 0], l[i, 1, 0]), (l[i, 0, 1], l[i, 1, 1]),
                zorder=1, c=lc[i], linewidth=lw, alpha=alpha)
            a.add_line(line)
        pts = l.reshape(-1, 2)
        a.scatter(pts[:, 0], pts[:, 1], c=point_color, s=ps,
                  linewidths=0, zorder=2, alpha=alpha)     
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)  # Remove spacing
    plt.show()

def shorten_lines(lines, shrink_factor=0.2):
    """
    Shortens all lines by a given percentage from both ends.

    Parameters:
    - lines (Nx2x2 array): Array of lines in the format [[x1, y1], [x2, y2]]
    - shrink_factor (float): Percentage to shorten from both ends (0.2 = 20%)

    Returns:
    - shortened_lines (Nx2x2 array): Array of shortened lines
    """
    if shrink_factor < 0 or shrink_factor > 1:
        raise ValueError("shrink_factor should be between 0 and 1")

    # Convert to float for precision
    lines = np.array(lines, dtype=np.float32)

    # Compute midpoints
    midpoints = (lines[:, 0, :] + lines[:, 1, :]) / 2

    # Move both endpoints closer to the midpoint
    new_endpoints_1 = midpoints + (lines[:, 0, :] - midpoints) * (1 - shrink_factor)
    new_endpoints_2 = midpoints + (lines[:, 1, :] - midpoints) * (1 - shrink_factor)

    # Stack the new endpoints into (N,2,2) shape
    shortened_lines = np.stack([new_endpoints_1, new_endpoints_2], axis=1)

    return shortened_lines.astype(np.int32)  # Convert back to int


def compute_slope(line, epsilon=1e-6):
    """Compute the slope of a line, handling vertical lines with epsilon."""
    (x1, y1), (x2, y2) = line
    return (y2 - y1) / (x2 - x1 + epsilon)  # Avoid division by zero

def compute_length(line):
    """Compute the Euclidean distance (length) of a line."""
    (x1, y1), (x2, y2) = line
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

import cv2
import numpy as np

def check_lines_in_mask(mask_path, lines):
    # Load binary mask image
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Ensure mask is binary (values 0 and 255)
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # Convert mask to a boolean array (True for inside, False for outside)
    mask_bool = mask > 0  # Pixels inside the mask are True

    # Get image dimensions
    height, width = mask.shape

    inside_both = []
    inside_one = []

    for line in lines:
        (x1, y1), (x2, y2) = line

        # Clip coordinates to ensure they are within image bounds
        x1 = np.clip(x1, 0, width - 1)
        y1 = np.clip(y1, 0, height - 1)
        x2 = np.clip(x2, 0, width - 1)
        y2 = np.clip(y2, 0, height - 1)

        # Check if both endpoints are inside the mask
        is_inside_1 = mask_bool[int(y1), int(x1)]
        is_inside_2 = mask_bool[int(y2), int(x2)]

        if is_inside_1 and is_inside_2:
            inside_both.append([[x1, y1], [x2, y2]])
        elif is_inside_1 or is_inside_2:
            inside_one.append([[x1, y1], [x2, y2]])

    # Convert lists to NumPy arrays with shape (N,2,2) and (M,2,2)
    inside_both = np.array(inside_both, dtype=np.int32) if inside_both else np.empty((0,2,2), dtype=np.int32)
    inside_one = np.array(inside_one, dtype=np.int32) if inside_one else np.empty((0,2,2), dtype=np.int32)

    return inside_both, inside_one


import numpy as np

def filter_short_lines(lines, image_height, min_ratio=0.01):
    """
    Remove lines whose length is less than a percentage of the image height.

    Args:
        lines (np.ndarray): Array of shape (N, 2, 2) representing line segments.
        image_height (int): Height of the image.
        min_ratio (float): Minimum length as a fraction of image height (default 1%).

    Returns:
        np.ndarray: Filtered lines with valid lengths.
    """
    if lines.size == 0:
        return lines  # Return empty if no lines exist

    min_length = min_ratio * image_height  # 1% of image height
    lengths = np.linalg.norm(lines[:, 1, :] - lines[:, 0, :], axis=1)  # Compute lengths
    filtered_lines = lines[lengths >= min_length]  # Keep only valid lines

    return filtered_lines

In [ ]:
# Model config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
conf = {
    'detect_lines': True,  # Whether to detect lines or only DF/AF
    'line_detection_params': {
        'merge': True,  # Whether to merge close-by lines
        'filtering': True,  # Whether to filter out lines based on the DF/AF. Use 'strict' to get an even stricter filtering
        'grad_thresh': 3,
        'grad_nfa': True,  # If True, use the image gradient and the NFA score of LSD to further threshold lines. We recommand using it for easy images, but to turn it off for challenging images (e.g. night, foggy, blurry images)
    }
}

# Load the model
ckpt = '../weights/deeplsd_md.tar'
ckpt = torch.load(str(ckpt), map_location='cpu')
net = DeepLSD(conf)
net.load_state_dict(ckpt['model'])
net = net.to(device).eval()

img_names = os.listdir(r"D:\3d-reconstruction\DeepLSD\room-downsam") #[0:1]
print(img_names)

for name in img_names:
    print(name)

    img_path = os.path.join(r"D:\3d-reconstruction\DeepLSD\room-downsam", name)

    # Load an image
    img = cv2.imread(img_path)[:, :, ::-1]
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Detect (and optionally refine) the lines
    inputs = {'image': torch.tensor(gray_img, dtype=torch.float, device=device)[None, None] / 255.}
    with torch.no_grad():
        out = net(inputs)
        pred_lines = out['lines'][0]

    print(pred_lines.shape)
    print(pred_lines[0])

    # # Plot the predictions
    # plot_images([img], ['All lines'], cmaps='gray')
    # plot_lines([pred_lines], indices=range(1))

    # Shorten the lines
    shortened = shorten_lines(pred_lines, shrink_factor=0.2)
    # plot_images([img], ['Shortened lines'], cmaps='gray')
    # plot_lines([shortened], indices=range(1))

    img_name = name.split(".")[0]
    print(img_name)
    mask_names = [name for name in os.listdir(rf"D:\3d-reconstruction\DeepLSD\masks\{img_name}_mask") if ".json" not in name]

    for mask_name in mask_names:

        mask_path = rf"D:\3d-reconstruction\DeepLSD\masks\{img_name}_mask\{mask_name}"
        inside_both, inside_one = check_lines_in_mask(mask_path, shortened)

        print("inside_both shape:", inside_both.shape)  # Expected: (N,2,2)
        print("inside_one shape:", inside_one.shape)    # Expected: (M,2,2)

        plot_images([img], ['Lines within Mask'], cmaps='gray')
        plot_lines([inside_both], indices=range(1))

        if inside_both.size != 0:


            import numpy as np
            from sklearn.cluster import DBSCAN

            directions = inside_both[:, 1, :] - inside_both[:, 0, :]  # Compute direction vectors
            norms = np.linalg.norm(directions, axis=1, keepdims=True)  # Compute norms
            directions_normalized = directions / norms  # Normalize direction vectors

            # Step 2: Compute pairwise angle differences in degrees
            dot_products = np.dot(directions_normalized, directions_normalized.T)  # Cosine similarity matrix
            dot_products = np.clip(dot_products, -1, 1)  # Ensure values stay within valid range
            angles = np.degrees(np.arccos(np.abs(dot_products)))  # Convert to absolute angle differences

            # Step 3: Use DBSCAN with an angle threshold
            angle_threshold = 5  # Maximum difference in degrees for parallel lines
            clustering = DBSCAN(eps=angle_threshold, min_samples=1, metric="precomputed").fit(angles)

            # Step 4: Group lines by clusters
            groups = {}
            group_lengths = {}

            for idx, label in enumerate(clustering.labels_):
                if label not in groups:
                    groups[label] = []
                    group_lengths[label] = 0  # Initialize total length for the group

                groups[label].append(inside_both[idx])
                
                # Compute and accumulate total length of the lines in the group
                line_length = np.linalg.norm(inside_both[idx, 1] - inside_both[idx, 0])  # Euclidean distance
                group_lengths[label] += line_length

            # Step 5: Select the best group based on the number of lines and total length
            best_group_label = max(groups.keys(), key=lambda lbl: (group_lengths[lbl], len(groups[lbl])))

            best_group = groups[best_group_label]
            best_group_length = group_lengths[best_group_label]

            # Compute distances for all lines in the best group
            line_distances = [np.linalg.norm(line[1] - line[0]) for line in best_group]

            # Find the index of the longest line
            longest_line_index = np.argmax(line_distances)

            # Get the longest line
            longest_line = best_group[longest_line_index]

            # Print results
            print(f"Best Group: {best_group_label}")
            longest_line = np.array(longest_line).reshape(1,2,2)
            plot_images([img], ['Lines within Mask'], cmaps='gray')
            plot_lines([longest_line], indices=range(1))








        # break

            # # Compute lengths and find the longest line
            # lengths = np.array([compute_length(line) for line in inside_both])

            # longest_index = np.argmax(lengths)
            # longest_line = inside_both[longest_index].reshape(1,2,2)

            # print(longest_line.shape)

            # plot_images([img], ['Longest line within Mask'], cmaps='gray')
            # plot_lines([longest_line], indices=range(1))

            # print(longest_line)

    # break